In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster        import KMeans
from sklearn.neural_network import MLPRegressor

import kagglehub
from kagglehub import KaggleDatasetAdapter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeRegressor

In [26]:
# Dataset Import
file_path = "vehicles.csv"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "austinreese/craigslist-carstrucks-data",
  file_path
)

print("First 5 records:", df.head())

/var/folders/s8/gtklhtj905s5w5st3lpb32mw0000gn/T/ipykernel_95054/494915325.py:4: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:            id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price  year  \
0                prescott    https://prescott.craigslist.org   6000   NaN   
1            fayetteville       https://fayar.craigslist.org  11900   NaN   
2            florida keys        https://keys.craigslist.org  21000   NaN   
3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
4              greensboro  https://greensboro.craigslist.org   4900   NaN   

  manufacturer model condition cylinders  ... size  type paint_color  \
0          NaN   NaN       NaN       Na

In [27]:
# Data Preprocessing
df = df.dropna(subset=['price', 'cylinders', 'year', 'odometer']) # Remove all rows with null values in any of the four categories

df = df[df['cylinders'] != "other"] # for some reason, some rows have a 'other' value for cylinders, so we shall remove these rows

df['cylinders'] = df['cylinders'].str[0].astype(int) # convert "8 cylinders" to the value 8, and so on for every num of cylinders

In [28]:
data = df[['price', 'year', 'odometer', 'cylinders']].copy()
X = data[['year', 'odometer', 'cylinders']]
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Matthew's Linear Regression

In [33]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

y_pred = model.predict(X_test_scaled)
print(f"R² score: {r2_score(y_test, y_pred):.4f}")
print(f"MSE:     {mean_squared_error(y_test, y_pred):.2f}")

print("Intercept:", model.intercept_)
print("Coefs:    ", dict(zip(X.columns, model.coef_)))

R² score: -0.0000
MSE:     594477277376181.75
Intercept: 35649.420754111474
Coefs:     {'year': np.float64(23610.01173775738), 'odometer': np.float64(-3475.37578709401), 'cylinders': np.float64(31640.170967601738)}


Logan's Polynomial Regression

In [30]:
# valuable LABELS: price, year, manufacturer, model, condition, cylinders, fuel, odometer,
#                  title_status, transmission, drive, size, type, paint_color, posting_date

# for now, use only the numerical ones:
#     price, year, cylinders, odometer


# Create polynomial features (degree=2)
poly = PolynomialFeatures(degree=2, include_bias=False)

# Transform the features
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Display the new feature names
feature_names = poly.get_feature_names_out(X.columns)
print("\nPolynomial feature names:")
print(feature_names)

# Create and train the polynomial regression model
poly_reg = LinearRegression()
poly_reg.fit(X_train_poly, y_train)

# Display the coefficients
print("\nModel coefficients:")
for i, coef in enumerate(poly_reg.coef_):
    print(f"{feature_names[i]}: {coef:.4f}")
print(f"Intercept: {poly_reg.intercept_:.4f}")

# Make predictions on the test set
y_pred = poly_reg.predict(X_test_poly)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nModel Evaluation:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.4f}")

# Create a DataFrame with actual and predicted prices
results = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

# Sort by actual price for better visualization
results = results.sort_values(by='Actual')

# Reset index
results = results.reset_index(drop=True)

def predict_car_price(year, cylinders, mileage):
    # Create a DataFrame with the new car's features
    new_car = pd.DataFrame({
        'year': [year],
        'cylinders': [cylinders],
        'odometer': [mileage]
    })

    # Transform the features to polynomial features
    new_car_poly = poly.transform(new_car)

    # Predict the price
    predicted_price = poly_reg.predict(new_car_poly)[0]

    return predicted_price



Polynomial feature names:
['year' 'odometer' 'cylinders' 'year^2' 'year odometer' 'year cylinders'
 'odometer^2' 'odometer cylinders' 'cylinders^2']

Model coefficients:
year: 0.0001
odometer: 2.0232
cylinders: 0.0053
year^2: 0.5548
year odometer: -0.0010
year cylinders: 10.6700
odometer^2: 0.0000
odometer cylinders: -0.0169
cylinders^2: 0.0618
Intercept: -2317892.0516

Model Evaluation:
Mean Squared Error (MSE): 594480923766981.12
Root Mean Squared Error (RMSE): 24381979.49
R-squared (R²): -0.0000


Dheeraj's Decision tree model:


In [31]:
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

Mean Squared Error: 594677716317001.5
R² Score: -0.00037773793882611706


everett's neural network

In [39]:
# Step 1: Apply K-Means clustering to the dataset
kmeans = KMeans(n_clusters=5, random_state=42)  # Choose 5 clusters (can be adjusted)
df['cluster'] = kmeans.fit_predict(X)  # Add cluster labels as a new feature

nn_model = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
nn_model.fit(X_train, y_train)

y_pred_nn = nn_model.predict(X_test)
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

print("\nNeural Network Evaluation:")
print(f"Mean Squared Error (MSE): {mse_nn:.2f}")
print(f"R-squared (R²): {r2_nn:.4f}")

# Predict car price using the neural network
def predict_price_nn(year, cylinders, mileage):
    cluster = kmeans.predict([[year, cylinders, mileage]])[0]  # Predict cluster for the new car
    new_car_nn = pd.DataFrame({
        'year': [year],
        'cylinders': [cylinders],
        'odometer': [mileage],
        'cluster': [cluster]
    })
    predicted_price_nn = nn_model.predict(new_car_nn)[0]
    return predicted_price_nn

# # Example usage
# year = 2018
# cylinders = 6
# mileage = 85000
# predicted_price_nn = predict_price_nn(year, cylinders, mileage)
# print(f"\nPredicted price for a car with {mileage} miles, {cylinders} cylinders, from year {year}: ${predicted_price_nn:.2f}")


Neural Network Evaluation:
Mean Squared Error (MSE): 594496634957787.38
R-squared (R²): -0.0001


In [37]:
# plt.figure(figsize=(10, 6))
# plt.plot(results.index, results['Actual'], label='Actual Prices', color='blue', marker='o')
# plt.plot(results.index, results['Predicted'], label='Predicted Prices', color='red', marker='x')
# plt.title('Actual vs. Predicted Car Prices')
# plt.xlabel('Index')
# plt.ylabel('Price')
# plt.legend()
# plt.grid(True)

# # Plot a scatter plot of actual vs. predicted prices
# plt.figure(figsize=(10, 6))
# plt.scatter(results['Actual'], results['Predicted'], alpha=0.5)
# plt.plot([results['Actual'].min(), results['Actual'].max()],
#          [results['Actual'].min(), results['Actual'].max()],
#          'k--', lw=2)
# plt.title('Actual vs. Predicted Car Prices')
# plt.xlabel('Actual Prices')
# plt.ylabel('Predicted Prices')
# plt.grid(True)
